In [211]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input, InputLayer, Dropout

import pandas as pd
import numpy as np

np.random.seed(42)

import os
import tensorflow as tf

#DATA NORMALIZATION(STANDARDISATION)
from sklearn.preprocessing import MinMaxScaler

#DATA PLOTTING
import matplotlib.pyplot as plt

#FOR PERFORMANCE METRICS ANALYSIS.
from keras.optimizers import Adam
from keras.losses import  MeanSquaredError
from keras.metrics import  RootMeanSquaredError

#SAVING AND LOADING MODEL
from keras.callbacks import ModelCheckpoint
from keras.models import load_model


In [212]:
nsc_data_frame = pd.read_csv("NSE_data_all_stocks_2022_jan_to_may (1).csv")
nsc_data_frame

,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,Change%,Volume,Adjusted Price
0,3-Jan-22,EGAD,Eaagads Ltd,10,15,13.5,13.8,13.5,13.5,-,-,"4,000",-
1,3-Jan-22,KUKZ,Kakuzi Plc,355,427,385,385,385,385,-,-,-,-
2,3-Jan-22,KAPC,Kapchorua Tea Kenya Plc,80,101,99.5,99.5,99.5,95.5,4,4.19%,100,-
3,3-Jan-22,LIMT,Limuru Tea Plc,260,360,320,320,320,320,-,-,-,-
4,3-Jan-22,SASN,Sasini Plc,16.75,22.6,18.7,18.7,18.7,18.7,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6727,31-May-22,MSC,Mumias Sugar Company Ltd,0.27,0.27,0.27,0.27,0.27,0.27,-,-,-,-
6728,31-May-22,UNGA,Unga Group Ltd,26.1,36.4,29,29,29,30,-1,3.33%,"2,100",-
6729,31-May-22,SCOM,Safaricom Plc,25.5,45.25,25.95,26.45,26,26.25,-0.25,0.95%,"20,079,900",-
6730,31-May-22,FAHR,Stanlib Fahari Income-REIT,5,7.48,5.5,5.6,5.56,5.58,-0.02,0.36%,"11,700",-


In [213]:
#DATA  CLEANING  # nsc_data_frame.set_index(nsc_data_frame.Date, inplace=True) to update the index  of the dataframe.

date_format = '%d-%b-%y'
nsc_data_frame.Date = pd.to_datetime(nsc_data_frame['Date'], format=date_format)
nsc_data_frame['Day_Of_The_Week'] = nsc_data_frame['Date'].dt.dayofweek

nsc_data_frame['Day_Name'] = nsc_data_frame['Date'].dt.day_name()

nsc_data_frame = nsc_data_frame.drop(columns=['Change', 'Change%', 'Volume', 'Adjusted Price'])


nsc_data_frame['Day_Sin'] = np.sin(2 * np.pi * nsc_data_frame['Day_Of_The_Week']/4.0)
nsc_data_frame['Day_Cos'] = np.cos(2 * np.pi * nsc_data_frame['Day_Of_The_Week']/4.0)

nsc_data_frame.sample(10)



,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Day_Of_The_Week,Day_Name,Day_Sin,Day_Cos
381,2022-01-10,KURV,Kurwitu Ventures Ltd,"1,500.00","1,500.00","1,500.00","1,500.00","1,500.00","1,500.00",0,Monday,0.000000e+00,1.000000e+00
2473,2022-02-23,SCAN,WPP ScanGroup Plc,3.5,6.36,4.37,4.4,4.38,4.4,2,Wednesday,1.224647e-16,-1.000000e+00
6214,2022-05-20,DTK,Diamond Trust Bank Kenya Ltd,48.5,69,48.5,51,49.95,50.5,4,Friday,-2.449294e-16,1.000000e+00
1559,2022-02-03,TOTL,Total Kenya Ltd,22.05,27.5,23.4,23.75,23.5,23.45,3,Thursday,-1.000000e+00,-1.836970e-16
2184,2022-02-17,CGEN,Car and General (K) Ltd,10,70,62,66,63.25,63,3,Thursday,-1.000000e+00,-1.836970e-16
2182,2022-02-17,SASN,Sasini Plc,16.75,22.6,21,21.85,21.5,20.95,3,Thursday,-1.000000e+00,-1.836970e-16
2638,2022-02-25,FAHR,Stanlib Fahari Income-REIT,5.5,7.48,6.2,6.3,6.22,6.22,4,Friday,-2.449294e-16,1.000000e+00
503,2022-01-12,TOTL,Total Kenya Ltd,22.05,27.5,24.5,25.3,24.5,24.2,2,Wednesday,1.224647e-16,-1.000000e+00
3341,2022-03-14,TOTL,Total Kenya Ltd,22.05,27.5,23.15,23.2,23.2,23.25,0,Monday,0.000000e+00,1.000000e+00
3206,2022-03-10,KPLC-P4,KPLC-P4,4.1,4.53,4.11,4.11,4.11,4.11,3,Thursday,-1.000000e+00,-1.836970e-16


In [214]:
#EGAD LTD.  DATA

egad_data_frame = nsc_data_frame[nsc_data_frame['Code'] == 'EGAD'].reset_index(drop=True)
egad_data_frame['Day Price'] = pd.to_numeric(egad_data_frame['Day Price'], errors='coerce')


# min_value = egad_data_frame['Day_Of_The_Week'].min()  ------ >> display(egad_data_frame.info())

display(egad_data_frame.head(10))

#SELECT THE 1ST COLUMN AND PLOT THE DAY_SINE AGAINST TIME

# plt.figure(figsize=(12, 6))  
# plt.subplot(1, 2, 1)  # Selects the first subplot
# egad_data_frame['Day_Sin'].plot()
# plt.title('Day_Sin Over Time')

#SELECT THE 2ND COLUMN AND PLOT THE DAY PRICE AGAINST TIME

# plt.subplot(1, 2, 2)  
# egad_data_frame['Day Price'].plot()
# plt.title('Day Price Over Time')
# 
# plt.tight_layout()  # To improve subplot spacing
# plt.show()




,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Day_Of_The_Week,Day_Name,Day_Sin,Day_Cos
0,2022-01-03,EGAD,Eaagads Ltd,10,15,13.5,13.8,13.50,13.5,0,Monday,0.000000e+00,1.000000e+00
1,2022-01-04,EGAD,Eaagads Ltd,10,15,12.85,12.85,12.85,13.5,1,Tuesday,1.000000e+00,6.123234e-17
2,2022-01-05,EGAD,Eaagads Ltd,10,15,12.85,12.85,12.85,12.85,2,Wednesday,1.224647e-16,-1.000000e+00
3,2022-01-06,EGAD,Eaagads Ltd,10,15,13.8,13.8,13.80,12.85,3,Thursday,-1.000000e+00,-1.836970e-16
4,2022-01-07,EGAD,Eaagads Ltd,10,15,13.8,13.8,13.80,13.8,4,Friday,-2.449294e-16,1.000000e+00
5,2022-01-10,EGAD,Eaagads Ltd,10,15,13.8,13.8,13.80,13.8,0,Monday,0.000000e+00,1.000000e+00
6,2022-01-11,EGAD,Eaagads Ltd,10,15,13.8,13.8,13.80,13.8,1,Tuesday,1.000000e+00,6.123234e-17
7,2022-01-12,EGAD,Eaagads Ltd,10,15,12.9,12.9,12.90,13.8,2,Wednesday,1.224647e-16,-1.000000e+00
8,2022-01-13,EGAD,Eaagads Ltd,10,15,12.9,12.9,12.90,12.9,3,Thursday,-1.000000e+00,-1.836970e-16
9,2022-01-14,EGAD,Eaagads Ltd,10,15,13.8,13.8,13.80,12.9,4,Friday,-2.449294e-16,1.000000e+00


In [215]:
df_interest = egad_data_frame[['Day Price', 'Day_Sin']].copy()

df_interest

,Day Price,Day_Sin
0,13.50,0.000000e+00
1,12.85,1.000000e+00
2,12.85,1.224647e-16
3,13.80,-1.000000e+00
4,13.80,-2.449294e-16
...,...,...
97,13.45,1.224647e-16
98,13.45,-1.000000e+00
99,13.45,-2.449294e-16
100,13.45,0.000000e+00


In [216]:
#Splitting the Dataframe into X_features and the Y_Classes
"""
WINDOW_SIZE = 5
[
  [[ds1, dp1], [ds2, dp2], [ds3, dp3], [ds4, dp4], [ds5, dp5]],
  [[ds2, dp2], [ds3, dp3], [ds4, dp4], [ds5, dp5], [ds6, dp6]]  
]
"""

def __generate_x_y_from_nsc_df(__data_frame , time_step=5):
    data_array = __data_frame.to_numpy()
    X = []
    y = []
    for i in range(len(data_array)-time_step):
        
        rows = data_array[i:i + time_step]   # print(rowing[1]) gives [12.85  1.] , that is the second row 
        
        zipped_row = [row for row in rows]  # creates a list of nd_arrays .
        
        label = data_array[i + time_step][0]
        X.append(zipped_row)
        y.append(label)
        
    return np.array(X), np.array(y)

X_data, Y_classes = __generate_x_y_from_nsc_df(df_interest , 5)

print("X_Data shape : ",X_data.shape)

print(X_data.shape , Y_classes.shape)

X_data[0]



X_Data shape :  (97, 5, 2)
(97, 5, 2) (97,)


array([[ 1.3500000e+01,  0.0000000e+00],
       [ 1.2850000e+01,  1.0000000e+00],
       [ 1.2850000e+01,  1.2246468e-16],
       [ 1.3800000e+01, -1.0000000e+00],
       [ 1.3800000e+01, -2.4492936e-16]])

In [217]:
#NORMALIZING DATA  Scaling  using the mean and std deviation : the second option is  to make sure that we normalize the data before preprocessing.

temp_training_mean = np.mean(X_data[:, :, 0])
temp_training_std = np.std(X_data[:, :, 0])


def normalize_prices(x_data_frame):
    prices_array  = x_data_frame[:, :, 0]    
    x_data_frame[:, :, 0] = (prices_array - temp_training_mean) / temp_training_std 
    return x_data_frame

normalized_data = normalize_prices(X_data)

normalized_data


    

array([[[ 8.68796552e-01,  0.00000000e+00],
        [-3.12774255e-01,  1.00000000e+00],
        [-3.12774255e-01,  1.22464680e-16],
        [ 1.41413692e+00, -1.00000000e+00],
        [ 1.41413692e+00, -2.44929360e-16]],

       [[-3.12774255e-01,  1.00000000e+00],
        [-3.12774255e-01,  1.22464680e-16],
        [ 1.41413692e+00, -1.00000000e+00],
        [ 1.41413692e+00, -2.44929360e-16],
        [ 1.41413692e+00,  0.00000000e+00]],

       [[-3.12774255e-01,  1.22464680e-16],
        [ 1.41413692e+00, -1.00000000e+00],
        [ 1.41413692e+00, -2.44929360e-16],
        [ 1.41413692e+00,  0.00000000e+00],
        [ 1.41413692e+00,  1.00000000e+00]],

       [[ 1.41413692e+00, -1.00000000e+00],
        [ 1.41413692e+00, -2.44929360e-16],
        [ 1.41413692e+00,  0.00000000e+00],
        [ 1.41413692e+00,  1.00000000e+00],
        [-2.21884193e-01,  1.22464680e-16]],

       [[ 1.41413692e+00, -2.44929360e-16],
        [ 1.41413692e+00,  0.00000000e+00],
        [ 1.41413692e+00

In [218]:
#Model Creation and Model Compilation.
dropout_rate = 0.2
input_shape=(normalized_data.shape[1], normalized_data.shape[2])

nsc_lstm_model = Sequential([Input(input_shape),
                    LSTM(units=100, return_sequences=True),
                    Dropout(rate=dropout_rate),
                    LSTM(units=100, return_sequences=False),
                    Dropout(rate= dropout_rate),
                    Dense(units=32, activation='relu'),
                    Dense(units = 1)])

nsc_lstm_model.compile(loss=MeanSquaredError(), 
              optimizer=Adam(learning_rate=0.0001),
              metrics=[RootMeanSquaredError()])

nsc_lstm_model.summary()


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_20 (LSTM)              (None, 100)               41200     
                                                                 
 dense_20 (Dense)            (None, 32)                3232      
                                                                 
 dense_21 (Dense)            (None, 1)                 33        
                                                                 
Total params: 44465 (173.69 KB)
Trainable params: 44465 (173.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [219]:

#++++++++++++++++++++++++++++++++++++++++++   SPLITTING DATA +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#

def dynamic_data_split(X, y, train_ratio, val_ratio, test_ratio):
    """
    Splits data and labels into training, validation, and test sets.

    Args:
        X (a 3D numpy.ndarray): Data.
        y (numpy.ndarray): Labels.
        train_ratio (float): Ratio for training data.
        val_ratio (float): Ratio for validation data.
        test_ratio (float): Ratio for test data.

    Returns:
        tuple: (X_train, y_train, X_val, y_val, X_test, y_test).
    """
    total_samples = X.shape[0]

    train_samples = int(train_ratio * total_samples)
    val_samples = int(val_ratio * total_samples)

    X_train, y_train = X[:train_samples], y[:train_samples]
    X_val, y_val = X[train_samples:train_samples + val_samples], y[train_samples:train_samples + val_samples]
    X_test, y_test = X[train_samples + val_samples:], y[train_samples + val_samples:]

    return X_train, y_train, X_val, y_val, X_test, y_test


train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

X_train, y_train, X_val, y_val, X_test, y_test = dynamic_data_split(normalized_data,Y_classes , train_ratio, val_ratio= val_ratio,test_ratio= test_ratio)

X_train.shape,y_train.shape, X_val.shape,y_val.shape, X_test.shape, y_test.shape


((77, 5, 2), (77,), (9, 5, 2), (9,), (11, 5, 2), (11,))

In [220]:
__nsc_model_checkpoint = ModelCheckpoint('nsc_model/', save_best_only=True)

nsc_lstm_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, callbacks=[__nsc_model_checkpoint])


Epoch 1/10
1/3 [=========>....................] - ETA: 6s - loss: 172.3184 - root_mean_squared_error: 13.1270INFO:tensorflow:Assets written to: nsc_model/assets


INFO:tensorflow:Assets written to: nsc_model/assets


3/3 [==============================] - 6s 2s/step - loss: 173.0906 - root_mean_squared_error: 13.1564 - val_loss: 147.0724 - val_root_mean_squared_error: 12.1273
Epoch 2/10
1/3 [=========>....................] - ETA: 0s - loss: 173.3066 - root_mean_squared_error: 13.1646INFO:tensorflow:Assets written to: nsc_model/assets


INFO:tensorflow:Assets written to: nsc_model/assets


3/3 [==============================] - 3s 1s/step - loss: 172.8283 - root_mean_squared_error: 13.1464 - val_loss: 146.5336 - val_root_mean_squared_error: 12.1051
Epoch 3/10
1/3 [=========>....................] - ETA: 0s - loss: 171.7030 - root_mean_squared_error: 13.1036INFO:tensorflow:Assets written to: nsc_model/assets


...

3/3 [==============================] - 3s 1s/step - loss: 171.4896 - root_mean_squared_error: 13.0954 - val_loss: 143.4451 - val_root_mean_squared_error: 11.9769
Epoch 9/10
1/3 [=========>....................] - ETA: 0s - loss: 170.0525 - root_mean_squared_error: 13.0404INFO:tensorflow:Assets written to: nsc_model/assets


INFO:tensorflow:Assets written to: nsc_model/assets


3/3 [==============================] - 3s 2s/step - loss: 171.2978 - root_mean_squared_error: 13.0881 - val_loss: 142.9456 - val_root_mean_squared_error: 11.9560
Epoch 10/10
1/3 [=========>....................] - ETA: 0s - loss: 171.5947 - root_mean_squared_error: 13.0994INFO:tensorflow:Assets written to: nsc_model/assets


INFO:tensorflow:Assets written to: nsc_model/assets


3/3 [==============================] - 3s 1s/step - loss: 171.1152 - root_mean_squared_error: 13.0811 - val_loss: 142.4572 - val_root_mean_squared_error: 11.9355
